---
title: Download biodiversity data
learning-goals:
  - Download data from the Global Biodiversity Information Facility (GBIF)
jupyter: python3
format:
  html: default
params:
  id: stars
  species_name: Tasiyagnunpa
  species_lookup: "sturnella neglecta"
  sample_filename: species-distribution-stars-data
  download_filename: gbif_tasiagnunpa
execute: 
  cache: false
---

## Access locations and times of Veery encounters

For this challenge, you will use a database called the [Global Biodiversity Information Facility (GBIF)](https://www.gbif.org/). GBIF is compiled from species observation data all over the world, and includes everything from museum specimens to photos taken by citizen scientists in their backyards.

::: {.callout-task title="Explore GBIF"}
Before your get started, go to the [GBIF occurrences search page](https://www.gbif.org/occurrence/search) and explore the data.
:::

::: {.callout-tip title="Contribute to open data"}

You can get your own observations added to GBIF using [iNaturalist](https://www.inaturalist.org/)!
:::

### Set up your code to prepare for download

We will be getting data from a source called [GBIF (Global Biodiversity Information Facility)](https://www.gbif.org/). We need a package called `pygbif` to access the data, which may not be included in your environment. Install it by running the cell below:

In [ ]:
#| output: false
%%bash
pip install pygbif

::: {.callout-task title="Import packages"}

In the imports cell, we've included some packages that you will need. Add imports for packages that will help you:

  1. Work with reproducible file paths
  2. Work with tabular data

:::

In [ ]:
import time
import zipfile
from getpass import getpass
from glob import glob

import pygbif.occurrences as occ
import pygbif.species as species

::: {.content-visible when-format="html"}

In [ ]:
import os
import pathlib
import time
import zipfile
from getpass import getpass
from glob import glob

import pandas as pd
import pygbif.occurrences as occ
import pygbif.species as species

:::

In [ ]:
# Create data directory in the home folder
data_dir = os.path.join(
    # Home directory
    pathlib.Path.home(),
    # Earth analytics data directory
    'earth-analytics',
    'data',
    # Project directory
    'your-project-directory-name-here',
)
os.makedirs(data_dir, exist_ok=True)

# Define the directory name for GBIF data
gbif_dir = os.path.join(data_dir, 'your-gbif-data-directory-name-here')

::: {.content-visible when-format="html"}

In [ ]:
# Create data directory in the home folder
data_dir = os.path.join(
    pathlib.Path.home(),
    'earth-analytics',
    'data',
    'migration',
)
os.makedirs(data_dir, exist_ok=True)

# Define the directory name for GBIF data
gbif_dir = os.path.join(data_dir, 'tasiyagnunpa_gbif_download')

:::

### Register and log in to GBIF

You will need a [GBIF account](https://www.gbif.org/) to complete this challenge. You can use your GitHub account to authenticate with GBIF. Then, run the following code to save your credentials on your computer.

::: {.callout-warning}
Your email address **must** match the email you used to sign up for GBIF!
:::

::: callout-tip
If you accidentally enter your credentials wrong, you can set `reset_credentials=True` instead of `reset_credentials=False`.
:::

In [ ]:
reset_credentials = False
# GBIF needs a username, password, and email
credentials = dict(
    GBIF_USER=(input, 'GBIF username:'),
    GBIF_PWD=(getpass, 'GBIF password'),
    GBIF_EMAIL=(input, 'GBIF email'),
)
for env_variable, (prompt_func, prompt_text) in credentials.items():
    # Delete credential from environment if requested
    if reset_credentials and (env_variable in os.environ):
        os.environ.pop(env_variable)
    # Ask for credential and save to environment
    if not env_variable in os.environ:
        os.environ[env_variable] = prompt_func(prompt_text)

### Get the species key

::: {.callout-task}

1. Replace the `species_name` with the name of the species you want to look up
2. Run the code to get the species key

:::

In [ ]:
# Query species
species_info = species.name_lookup(species_name, rank='SPECIES')

# Get the first result
first_result = species_info['results'][0]

# Get the species key (speciesKey)
species_key = first_result['speciesKey']

# Check the result
first_result['species'], species_key

::: {.content-visible when-format="html"}

In [ ]:
# Query species
species_info = species.name_lookup("sturnella neglecta", rank='SPECIES')

# Get the first result
first_result = species_info['results'][0]

# Get the species key (speciesKey)
species_key = first_result['speciesKey']

# Check the result
first_result['species'], species_key

:::

### Download data from GBIF

::: {.callout-task title="Submit a request to GBIF"}

1. Replace `csv_file_pattern` with a string that will match **any** `.csv` file when used in the `glob` function. HINT: the character `*` represents any number of any values except the file separator (e.g. `/`)
2. Add parameters to the GBIF download function, `occ.download()` to limit your query to:

    - observations
    - from 2023
    - with spatial coordinates.

3. Then, run the download. **This can take a few minutes**.
:::

In [ ]:
# Only download once
gbif_pattern = os.path.join(gbif_dir, csv_file_pattern)
if not glob(gbif_pattern):
    # Submit query to GBIF
    gbif_query = occ.download([
        "speciesKey = ",
        "year = ",
        "hasCoordinate = ",
    ])
    # Only download once
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]

        # Wait for the download to build
        wait = occ.download_meta(download_key)['status']
        while not wait=='SUCCEEDED':
            wait = occ.download_meta(download_key)['status']
            time.sleep(5)

    # Download GBIF data
    download_info = occ.download_get(
        os.environ['GBIF_DOWNLOAD_KEY'], 
        path=data_dir)

    # Unzip GBIF data
    with zipfile.ZipFile(download_info['path']) as download_zip:
        download_zip.extractall(path=gbif_dir)

# Find the extracted .csv file path
gbif_path = glob(gbif_pattern)[0]

::: {.content-visible when-format="html"}

In [ ]:
# Only download once
gbif_pattern = os.path.join(gbif_dir, '*.csv')
if not glob(gbif_pattern):
    # Only submit one request
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:
        # Submit query to GBIF
        gbif_query = occ.download([
            f"speciesKey = { species_key }",
            "hasCoordinate = TRUE",
            "year = 2024",
        ])
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]

    # Wait for the download to build
    download_key = os.environ['GBIF_DOWNLOAD_KEY']
    wait = occ.download_meta(download_key)['status']
    while not wait=='SUCCEEDED':
        wait = occ.download_meta(download_key)['status']
        time.sleep(5)

    # Download GBIF data
    download_info = occ.download_get(
        os.environ['GBIF_DOWNLOAD_KEY'], 
        path=data_dir)

    # Unzip GBIF data
    with zipfile.ZipFile(download_info['path']) as download_zip:
        download_zip.extractall(path=gbif_dir)

# Find the extracted .csv file path (take the first result)
gbif_path = glob(gbif_pattern)[0]

:::

### Load the GBIF data into Python

::: {.callout-task title="Load GBIF data"}

1.  Look at the beginning of the file you downloaded using the code below. What do you think the **delimiter** is?
2.  Run the following code cell. What happens?
3.  Uncomment and modify the parameters of `pd.read_csv()` below until your data loads successfully and you have only the columns you want.
:::

You can use the following code to look at the beginning of your file:

In [ ]:
!head -n 2 $gbif_path 

In [ ]:
# Load the GBIF data
gbif_df = pd.read_csv(
    gbif_path, 
    #delimiter='',
    #index_col='',
    #usecols=[]
)
gbif_df.head()

::: {.content-visible when-format="html"}

In [ ]:
# Load the GBIF data
gbif_df = pd.read_csv(
    gbif_path, 
    delimiter='\t',
    index_col='gbifID',
    usecols=['gbifID', 'decimalLatitude', 'decimalLongitude', 'month'])
gbif_df.head()

:::

::: {.content-hidden}

In [ ]:
#| eval: true
import os
import shutil
import subprocess

import requests

try:
    file_name = "__param_sample_filename.zip"
    shutil.copy2(download_info['path'], file_name)

    subprocess.run([
        'gh', 'release', 'upload', 'data-release',
        file_name,
        '--repo', 'cu-esiil-edu/esiil-learning-portal',
        '--clobber'
        ])
except NameError:
    print('Looks like the current version of data has already been uploaded.')

:::
